In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../')

In [ ]:
from src.utils import loading, Spark
import pyspark.ml as M
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [ ]:
spark

In [ ]:
datas = loading(spark, '../downloads')

In [ ]:
datas

In [ ]:
ratings = datas['ratings']


In [ ]:
for key in datas.keys():
    print(key)
    datas[key].show(10)

In [ ]:
movies = datas['movies']

In [ ]:
movies.show()

In [ ]:
movies.select('genres')

In [ ]:
rating_user_count = ratings.groupby('userId').agg(
            F.count(F.col('movieId')).alias('movie_count')
)

rating_item_count = ratings.groupby('movieId').agg(
            F.count(F.col('userId')).alias('user_count')
)

In [ ]:
userid = rating_user_count.sort(F.col('movie_count').desc()).limit(20000)

In [ ]:
itemid = rating_item_count.sort(F.col('user_count').desc()).limit(1000)

In [ ]:
itemid

In [ ]:
sample = ratings.join(userid, ratings.userId == userid.userId, 'inner')\
                .select(ratings.userId, ratings.movieId, ratings.rating)\
                .join(itemid, ratings.movieId == itemid.movieId, 'inner')\
                .select(ratings.userId, ratings.movieId, ratings.rating)

In [ ]:
sample

In [ ]:
sample.select('userId').distinct().count()

In [ ]:
sample.select('movieId').distinct().count()

In [ ]:
sample.count()

In [ ]:
stringIndexer = M.feature.StringIndexer(inputCol='userId', outputCol='userId_indx')
model = stringIndexer.fit(sample)
sample = model.transform(sample)
stringIndexer = M.feature.StringIndexer(inputCol='movieId', outputCol='movieId_indx')
model = stringIndexer.fit(sample)
sample = model.transform(sample)

In [ ]:
sample = sample.select('userId', 'movieId', \
                      F.col('userId_indx').cast(T.IntegerType()),
                      F.col('movieId_indx').cast(T.IntegerType()),
                    'rating')\
               .orderBy(['userId_indx', 'movieId_indx'])

In [ ]:
sample.coalesce(1).write.csv(path=fp, compression='gzip', mode='overwrite', header = True)